In [76]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
df = pd.read_csv('clean_book.csv', encoding='unicode_escape', error_bad_lines=False)
df_copy = df.copy()
df_copy

C:\Users\baiba\AppData\Local\Temp\ipykernel_3592\2066937374.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('clean_book.csv', encoding='unicode_escape', error_bad_lines=False)


,Unnamed: 0,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,day,year,real_author
0,0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,9,16,2006,J.K. Rowling
1,1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,9,1,2004,J.K. Rowling
2,2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,11,1,2003,J.K. Rowling
3,3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,5,1,2004,J.K. Rowling
4,4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,9,13,2004,J.K. Rowling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11122,11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,2004-12-21,Da Capo Press,12,21,2004,William T. Vollmann
11123,11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9780140110876,eng,635,783,56,1988-12-01,Penguin Books,12,1,1988,William T. Vollmann
11124,11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,140131965,9780140131963,eng,415,820,95,1993-08-01,Penguin Books,8,1,1993,William T. Vollmann
11125,11125,45639,Poor People,William T. Vollmann,3.72,60878827,9780060878825,eng,434,769,139,2007-02-27,Ecco,2,27,2007,William T. Vollmann


In [79]:
columns = ['title', 'real_author', 'publisher']

In [80]:
def combine_features(data):
    features=[]
    for i in range(0, data.shape[0]):
        features.append(data['title'][i]+ ' ' + data['real_author'][i] + ' ' + data['publisher'][i])
        
    return features

In [81]:
df['combined_features']=combine_features(df)
df

,Unnamed: 0,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,day,year,real_author,combined_features
0,0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,9,16,2006,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...
1,1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,9,1,2004,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...
2,2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,11,1,2003,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...
3,3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,5,1,2004,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...
4,4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,9,13,2004,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11122,11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,2004-12-21,Da Capo Press,12,21,2004,William T. Vollmann,Expelled from Eden: A William T. Vollmann Read...
11123,11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9780140110876,eng,635,783,56,1988-12-01,Penguin Books,12,1,1988,William T. Vollmann,You Bright and Risen Angels William T. Vollman...
11124,11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,140131965,9780140131963,eng,415,820,95,1993-08-01,Penguin Books,8,1,1993,William T. Vollmann,The Ice-Shirt (Seven Dreams #1) William T. Vol...
11125,11125,45639,Poor People,William T. Vollmann,3.72,60878827,9780060878825,eng,434,769,139,2007-02-27,Ecco,2,27,2007,William T. Vollmann,Poor People William T. Vollmann Ecco


In [82]:
new_df = df[['bookID','title','combined_features']]
new_df

,bookID,title,combined_features
0,1,Harry Potter and the Half-Blood Prince (Harry ...,Harry Potter and the Half-Blood Prince (Harry ...
1,2,Harry Potter and the Order of the Phoenix (Har...,Harry Potter and the Order of the Phoenix (Har...
2,4,Harry Potter and the Chamber of Secrets (Harry...,Harry Potter and the Chamber of Secrets (Harry...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,Harry Potter and the Prisoner of Azkaban (Harr...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,Harry Potter Boxed Set Books 1-5 (Harry Potte...
...,...,...,...
11122,45631,Expelled from Eden: A William T. Vollmann Reader,Expelled from Eden: A William T. Vollmann Read...
11123,45633,You Bright and Risen Angels,You Bright and Risen Angels William T. Vollman...
11124,45634,The Ice-Shirt (Seven Dreams #1),The Ice-Shirt (Seven Dreams #1) William T. Vol...
11125,45639,Poor People,Poor People William T. Vollmann Ecco


In [83]:
vectors = CountVectorizer().fit_transform(new_df['combined_features']).toarray()

In [88]:
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.80295507, 0.77459667, ..., 0.08333333, 0.        ,
        0.        ],
       [0.80295507, 1.        , 0.82928843, ..., 0.15294382, 0.        ,
        0.        ],
       [0.77459667, 0.82928843, 1.        , ..., 0.0860663 , 0.        ,
        0.        ],
       ...,
       [0.08333333, 0.15294382, 0.0860663 , ..., 1.        , 0.2981424 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.2981424 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [94]:
def recommend(book):
    
    book_index = new_df[new_df['title']==book].index[0]
    distances = similarity[book_index]
    books_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in books_list:
        print(new_df.iloc[i[0]].title)
    

In [103]:
recommend('Atonement')

Enduring Love
The Cement Garden
Saturday
Black Dogs
The Child in Time
The Comfort of Strangers
Amsterdam
Vintage Murakami
The Quiet American
Possession
